<a href="https://colab.research.google.com/github/Mirza-Ismail-Beg/Image-Analysis/blob/main/Image%20Caption%20Generator/Major_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import string 
import numpy as np
from PIL import Image
import os
from pickle import dump,load

from keras.applications.xception import Xception, preprocess_input
from keras.preprocessing.image import load_img , img_to_array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers.merge import add
from keras.layers import Input , Dense , LSTM , Embedding , Dropout
from keras.models import Model , load_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import plot_model

from tqdm import tqdm_notebook as tq
tq().pandas()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


0it [00:00, ?it/s]

In [ ]:
#Uploading text file
def upload(filename):
  file = open(filename,'r')
  text = file.read()
  file.close()
  return text

#get all image 
def img_caption(filename):
  file = upload(filename)
  captions = file.split('\n')
  des = {}
  for caption in captions[:-1]:
    img, caption = caption.split('\t')
    if img[:-2] not in des:
      des [img[:-2]] = [caption]
    else:
      des[img[:-2]].append(caption)    
  return des

def remove(captions):
  table = str.maketrans('','',string.punctuation)
  for img,caps in captions.items():
    for i,img_caption in enumerate(caps):
      img_caption.replace("-"," ")
      desc = img_caption.split()
      desc = [word.lower() for word in desc]                                  #converts to lowercase
      desc = [word.translate(table) for word in desc]                         #remove punctuation from each token 
      desc = [word for word in desc if(len(word)>1)]                          #remove hanging 's and a
      desc = [word for word in desc if(word.isalpha())]                       #remove tokens with numbers in them

      img_caption = ' '.join(desc)                                            #convert back to string
      captions[img][i]= img_caption
            
  return captions  

def text(des):
  get = set()
  for key in des.keys():
    [get.update(d.split()) for d in des[key]]
  
  return get  

def update_des(des,filename):
  l = list()
  for key, desc_list in des.items():
    for i in desc_list:
      l.append(key + '\t' + i)
  
  data = "," + '\n'.join(l)
  file = open(filename,'w')
  file.write(data)
  file.close()

dataset_txt = "/content/Flickr8k.token.txt"
from google.colab import drive
drive.mount('/content/drive')
des =  img_caption(dataset_txt)                                #mapping des distionary img to 5 captions
#print(des)
rem = remove(des)                                              #cleaning the des
txt = text(rem)
print("lengh of txt =" , len(rem))                             #bluiding text
update = update_des(rem,"des.txt")                             #update the des into file

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
lengh of txt = 8092


In [ ]:
#dataset_img = "/content/drive/MyDrive/Flicker8k_Dataset"
#def extract(directory):
#  model = Xception(include_top=False , pooling='avg')
#  features={}
#  for img in tq(os.listdir(directory)):
#    filename = directory + '/' + img
#    image = Image.open(filename)
#    image = image.resize((299,299))
#    image = np.expand_dims(image , axis=0)
#    image = image/127.5   
#    image = image-1.0 

#    f = model.predict(image)
#    features[img] = f

#  return features   

#extract_feature = extract(dataset_img)
#dump(extract_feature , open("features.p","wb"))

In [ ]:
#loading the data
def load_pic(filename):
  file = upload(filename)
  pic = file.split("\n")[:-1]
  return pic

def create(filename , pic):
  file = upload(filename)
  des = {}
  for line in file.split("\n"):
    words = line.split()
    if len(words)<1:
      continue           
    
    image, image_caption = words[0], words[1:]
    if image in pic:
      if image not in des:
        des[image] = []
      desc = '<start> ' + " ".join(image_caption) + ' <end>'
      des[image].append(desc)
    
  return des

def feature(pic):
  features = "/content/feature.p"
  all_features = load(open(features,"rb"))                     #for loading all the features
  features = {k:all_features for k in pic}                     #for selecting only needed features
  return features

dataset_trainImages = "/content/Flickr_8k.trainImages.txt"
t_images = load_pic(dataset_trainImages)
t_des = create("des.txt",t_images)
t_feature = feature(t_images)  

In [ ]:
#converting dictionary to the list
def to_list(des):          
  desc = []
  for key in des.keys():
    [desc.append(d) for d in des[key]]   

  return desc  

#Creating a tokenizer class . So that each integer represent the tokens in the dictionary 
def create_token(des):
  li = to_list(des)
  token = Tokenizer()
  token.fit_on_texts(li)
  return token

tokens = create_token(t_des)
dump(tokens,open("tokens.p","wb"))
size = len(tokens.word_index) + 1

def max(des):
  desc = to_list(des)
  max = 0
  for d in desc:
    l = len(d.split())
    if (l>max):
      max=l
  return max

max_length = max(des) 

In [ ]:
def generate(des,features,token,maxi):
  while 1:
    for key,des_list in des.items():
      feature = features[key][0]
      image , sequence , wording = create_seq(tokens,max_length,des_list,feature)
      yield [[image,sequence], wording]

def create_seq(tokens,max_length,des_list,feature):
  x,y,z = list(), list(), list()
  for d in des_list:
    sq = tokens.texts_to_sequences([d])[0]                                  #for encoding the sequences
    l = len(sq)
    for i in range(1,l):
      a,b = sq[:i], sq[i]
      a = pad_sequences([a], maxlen=max_length)[0]                          #for padding input sequence
      b = to_categorical([b], num_classes = size)[0]                           #for encoding the ouput sequence
      x.append(feature)
      y.append(a)
      z.append(b)

  return np.array(x),np.array(y),np.array(z)

feature_p = "/content/feature.p"
all = load(open(feature_p,"rb"))
[a,b],c = next(generate(t_des, all , tokens, max_length))      
print(a.shape , b.shape , c.shape)

(32, 2048) (32, 32) (32, 7576)


In [ ]:
def model(size,length):
  n1 = Input(shape=(2048,))                                      #Squeezing features from NN model from 2048 to 256
  a1 = Dropout(0.5)(n1)
  b1 = Dense(256, activation='relu')(a1)
  
  n2 = Input(shape=length,)                                      #LTSM sequence model
  a2 = Embedding(size , 256 , mask_zero = True)(n2)
  b2 = Dropout(0.5)(a2)
  c2 = LSTM(256)(b2)

  d1 = add([b1,c2])
  d2 = Dense(256 , activation='relu')(d1)
  o1 = Dense(size , activation='softmax')(d2)

  m = Model(inputs=(n1,n2),outputs=o1)
  m.compile(loss='categorical_crossentropy', optimizer='adam')

  print(m.summary())
  plot_model(m, to_file='model.png', show_shapes=True)

  return m

print("length of des =" , len(des))
print("Dataset = ",len(t_images)) 
print("Des train = ",len(t_des))
print("Pic train = ",len(t_feature))
print("len of vocabulary = ",size)
print("Maximum length = ",max_length)
mo = model(size, max_length)
epochs = 10
s = len(t_des)
os.mkdir("models")
for i in range(epochs):
  generator = generate(t_des,t_feature,tokens,max_length)
  #mo.fit_generator(generator, epochs=1, steps_per_epoch= s, verbose=1)
  #mo.save("models/mo_" + str(i) + ".h5")
  print(generator)  